1. Загрузка библиотек
2. Создание папки row_data для выгрузки сырых данных
3. Проверка последней даты в базе данных
4. Выгрузка сырых данных в папку row_data
5. Обработка данных
6. Выгрузка данных в базу
7. Удаление файла из папки row_data

In [5]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 25.7 MB/s eta 0:00:00


In [6]:
import os
import pandas as pd
import requests
import datetime
import sqlalchemy
import json
import sys
from time import sleep
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus # для экранирования спецсимволов: ! * %

In [7]:
# создаём папку (если её ранее не было) для выгрузки сырых данных из Директа
row_data = 'row_data'

path = os.path.join(os.getcwd(), row_data)
if not os.path.exists(path):
  os.mkdir(row_data)


In [8]:
# Адрес сервиса Reports для отправки JSON-запросов
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

### Создание подключения к базе данных

In [9]:
# Параметры подключения
host = "actual_host"
port = 3306
user = "actual_user"
password = "actual_password"
database = "actual_database"

# Кодируем пароль
encoded_password = quote_plus(password)

# Создаем строку подключения
connection_string = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}:{port}/{database}"

try:
    engine = create_engine(connection_string)
    with engine.connect() as connection:
        print('Подключение успешно установлено!')
except Exception as e:
  with open('log.txt', 'a') as f:
    print(f' при подключении к базе данных, Error: {e}')


Подключение успешно установлено!


### Функция для запроса данных из базы

In [10]:
# Функция для упрощения запросов к базе
def db_query(query):
    with engine.connect() as connection:
        return pd.read_sql(query, connection)

In [ ]:
# Получаем максимальную дату, по которым есть данные в базе
query = """ Select max(Date)
From allopizza_yd_jetstat"""

max_date = db_query(query)

# Определяем даты для загрузки данных
yesterday = datetime.date.today() - datetime.timedelta(days = 1)

flag = False # флаг для определения наличия новых данных для выгрузки

if yesterday < max_date.iloc[0,0]:
  flag = True # флаг для определения наличия новых данных для выгрузки
  dateS = datetime.datetime.strftime((max_date.iloc[0,0] + datetime.timedelta(days = 1)), format) # дата начала выгружаемого периода start date
  dateE = datetime.datetime.strftime(yesterday, format) # дата окончания выгружемого периода end date
  print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")} Есть новые данные для выгрузки')
else:
  print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")} Нет новых данных для выгрузки')

# Вспомогательные функции

# Функция для создания запроса к API Direct

In [43]:
# Входные данные
# Адрес сервиса Reports для отправки JSON-запросов
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

# Функция для создания запроса
def create_request(token:str, сlientLogin:str, dateS:str, dateE:str):

  #  Подготовка к запросу
  #  Создание заголовка HTTP запроса
  headers = {
      # OAuth-токен. Слово "Bearer" обязательно к использованию
      "Authorization": "Bearer " + token,
      # Логин клиента рекламного агентства
      "Client-Login": сlientLogin,
      # Язык ответных сообщений
      "Accept-Language": "en",
      # Режим формирования отчета: online, offline или auto.
      "processingMode": "auto",
      # Формат денежных значений в отчете
      "returnMoneyInMicros": "false",
      # Не выводить в отчете строку с названием отчета и диапазоном дат
      "skipReportHeader": "true",
      # Не выводить в отчете строку с названиями полей.
      "skipColumnHeader": "false",
      # Не выводить в отчете строку с количеством строк статистики.
      "skipReportSummary": "true"
  }

  # Создание тела запроса

  body = {
          "params": {
              "SelectionCriteria": {
                  "DateFrom": dateS,
                  "DateTo": dateE
              },
              "Goals": ["3030778008", "311574500", "311574248" ], # укажите номера целей из Метрики
              "AttributionModels": ["LSC"],
              "FieldNames": [
                  "Date",
                  "CampaignName",
                  "CampaignId",
                  "Criterion",
                  "AdNetworkType",
                  "AdFormat",
                  "Device",
                  "Placement",
                  "Slot",
                  "Impressions",
                  "Clicks",
                  "Cost",
                  "AvgClickPosition",
                  "AvgImpressionPosition",
                  "Conversions",
              ],
              "ReportName": "Request6" + dateS,
              "Page": {
                  "Limit": 10000000
              },
              "ReportType": "CUSTOM_REPORT",
              "DateRangeType": "CUSTOM_DATE",
              "Format": "TSV",
              "IncludeVAT": "YES",
              "IncludeDiscount": "NO"
          }
      }

  body = json.dumps(body, indent=4)
  return body, headers

#### Функция для отправки запроса к API Direct

In [44]:
# Функция для отправки запроса
def send_request(token:str, сlientLogin:str, clientName,  dateS:str, dateE:str,path=path):
  """ token - токен полученный в Яндексе (нужно создать приложение в Яндексе и далее по инструкци),
  client_login -  логин клиента в Директе, clientName - можете дать любое название того, что выгружаете
  """

  body, headers = create_request(token,сlientLogin, dateS,dateE)

  # Цикл передачи запроса и обработка ответа от сервера
  while True:
      try:
          req = requests.post(ReportsURL, body, headers=headers)
          req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
          if req.status_code == 400:
              #print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
              #print("RequestId: {}".format(req.headers.get("RequestId", False)))
              #print("JSON-код запроса: {}".format(u(body)))
              #print("JSON-код ответа сервера: \n{}".format(u(req.json())))
              break
          elif req.status_code == 200:
              #print("Отчет создан успешно")
              #print("RequestId: {}".format(req.headers.get("RequestId", False)))
              # print("Содержание отчета: \n{}".format(u(req.text)))
              break
          elif req.status_code == 201:
              #print("Отчет успешно поставлен в очередь в режиме офлайн")
              retryIn = int(req.headers.get("retryIn", 60))
              #print("Повторная отправка запроса через {} секунд".format('60'))
              #print("RequestId: {}".format(req.headers.get("RequestId", False)))
              sleep(60)
          elif req.status_code == 202:
              #print("Отчет формируется в режиме офлайн")
              retryIn = int(req.headers.get("retryIn", 60))
              #print("Повторная отправка запроса через {} секунд".format(retryIn))
              #print("RequestId:  {}".format(req.headers.get("RequestId", False)))
              sleep(retryIn)
          elif req.status_code == 500:
              #print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
              #print("RequestId: {}".format(req.headers.get("RequestId", False)))
              #print("JSON-код ответа сервера: \n{}".format(u(req.json())))
              break
          elif req.status_code == 502:
              #print("Время формирования отчета превысило серверное ограничение.")
              #print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
              #print("JSON-код запроса: {}".format(body))
              #print("RequestId: {}".format(req.headers.get("RequestId", False)))
              #print("JSON-код ответа сервера: \n{}".format(u(req.json())))
              break

          else:
              #print("Произошла непредвиденная ошибка")
              #print("RequestId:  {}".format(req.headers.get("RequestId", False)))
              #print("JSON-код запроса: {}".format(body))
              #print("JSON-код ответа сервера: \n{}".format(u(req.json())))
              break

  # Обработка ошибки, если не удалось соединиться с сервером API Директа
      except ConnectionError as e:
          # В данном случае мы рекомендуем повторить запрос позднее
          #print("Произошла ошибка соединения с сервером API")
          # Принудительный выход из цикла
          break

          # Если возникла какая-либо другая ошибка
      except Exception as e:
          # В данном случае мы рекомендуем проанализировать действия приложения
          #print("Произошла непредвиденная ошибка")
          # Принудительный выход из цикла
          break

  # Запись данных в файл формата TSV
  with open((os.path.join(path, clientName + '_' + dateS + '-' + dateE + '.tsv')), 'w', encoding="utf-8") as f:
    f.write(req.text)

  # Запись полученного файла с данными в датафрейм df1
  df1 = pd.read_csv(os.path.join(path, clientName + '_' + dateS + '-' + dateE + '.tsv'), sep="\t",
                    dtype={"AdGroupId": str})


  return df1

#### Функция для преобразования исходного датафрема в нужный формат для загрузки в базу данных

In [45]:
#### df1 to df2 transoform
def df_prepare(df1):
  df2 = pd.DataFrame({
      'Date': df1['Date'],
      'CampaignName': df1['CampaignName'],
      'CampaignId': df1['CampaignId'],
      'Criterion': df1['Criterion'],
      'AdNetworkType': df1['AdNetworkType'],
      'AdFormat':df1['AdFormat'],
      'Device': df1['Device'],
      'Placement': df1['Placement'],
      'Slot': df1['Slot'],
      'Impressions': df1['Impressions'],
      'Clicks': df1['Clicks'],
      'Cost': df1['Cost'],
      'AvgClickPosition': df1['AvgClickPosition'],
      'AvgImpressionPosition': df1['AvgImpressionPosition'],
      'Ecommerce_purchase_3030778008': df1['Conversions_3030778008_LSC'],
      'Call_calltouch_311574500': df1['Conversions_311574500_LSC'],
      'Unique_call_calltouch_': df1['Conversions_311574248_LSC']})
  return df2

#### Выгрузка данных Директа в папку row_data

In [ ]:
# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = 'actual_token'
clientLogin = 'Direct_login_actual' # догин клиента в аккаунте Директа
clientName = 'any_name_you_want'

if flag:
  try:
    df = send_request(token, clientLogin, clientName, dateS, dateE, path=path)
    print('Данные по клиету загружены на диск')
  except:
    df = send_request(token, clientLogin, clientName, dateS, dateE, path=path) # повтрный запрос, если первый не прошёл
  else:
    print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")}, Данные загрузить не удалось.')


#### Функция для отправки данных в базу данных

In [51]:
def perform_db_task(df, table_name: str): # функция для загрузки данных с диска в базу
  """ table_name - название таблицы в базе данных"""
  try:
    df.to_sql(table_name, con=engine, if_exists='append', index=False, chunksize=1000)
    print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")} Данные {table_name} успешно добавлены.', file=f)
  except Exception as e:
    print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")} Ошибка при добавлении данных в базу: {e}', file=f)

#### Выгрузка из папки row_date в базу

In [ ]:
# Выгрузка в базу  из папки
files = os.listdir(path)
table_name = 'db_table_name' # название таблицы в базе данных, которая существует или которую хотите создать

for i in files:
  df = pd.read_csv(os.path.join(path, i), sep='\t')
  df = df_prepare(df)
  try:
    perform_db_task(df, table_name)
    print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")}, Выгрузка данных из файла {i} в базу прошла с первого раза', file=f)
  except:
    try:
      perform_db_task(df, table_name)
      print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")}, Выгрузка данных из файла {i} в базу прошла со второго раза', file=f)
    except Exception as e:
      print(f'Current time: {datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")},!! Выгрузка данных из файла {i} в базу не прошла,  Error: {e}', file=f)


#### Удаление сырых данных из папки row_data, если необходимо

In [ ]:
for i in os.listdir(path):
    file_path = os.path.join(path, i)
    if os.path.isfile(file_path):
        os.remove(file_path)